In [9]:
%matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter, StrMethodFormatter, FormatStrFormatter

Using matplotlib backend: TkAgg


In [2]:
# Henry's law
def calc_hcp(T):
    C = 2400 
    T0 = 298.15
    hcp0 = 3.4e-2
    return hcp0*np.exp(C*(1/T - 1/T0))

hcp = calc_hcp(T=273.15 + 37)

def qf(a,b,c):
    return (-b+np.sqrt(b**2-4*a*c))/(2*a)

def ph_to_co2(ph, hco3, hcp=hcp):
    h = 10**(-ph)
    w = 1e-7
    ka = 4.45e-7
    bc = hco3 + h - w
    co2 = bc * h / ka
    return co2 / hcp

def general_ph(c1, c2, c3, ka):
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

def cpH(co2, bicarb, hcp=hcp):
    # co2 as fraction of atmosphere
    c1 = co2 * hcp # henry's law
    c2 = 1e-7
    c3 = bicarb
    ka = 4.45e-7  # pka 6.35
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

In [51]:
fig = plt.figure()

In [52]:
try:
    fig.clear()
except:
    fig = plt.figure()

co2 = np.arange(0, 101)
bc = 2.02
ax = fig.add_subplot(1,1,1)
for bc in (1, 2.02, 3.7, 4.5):

    # (1-co2/100) * 0.0004 should account for co2 in air
    # assuming no other gasses flow. This value disappears into
    # rounding error very quickly. 
    ph = cpH(co2/100 + (1-co2/100)*0.0004, bc/84)
    ax.plot(co2, ph, label="%.2f g/L HCO\u2083"%bc)

xax = ax.xaxis
xax.set_label_text("CO\u2082 %")
xax.set_major_formatter(FormatStrFormatter("%d%%"))

yax = ax.yaxis
yax.set_label_text("pH")
yax.set_major_formatter(FormatStrFormatter("%.2f"))

ax.set_xlim(0, 100)
ax.set_ylim(6, 8)

ax.grid()
ax.legend()

fig.suptitle("CO\u2082 vs. pH for different HCO\u2083 concentrations")


fig.show()